BİRADS KODU BURDAN BAŞLIYOR


In [5]:
!pip install nltk


In [15]:
!pip install imbalanced-learn


In [ ]:
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

nltk.download('stopwords')

stop_words = set(stopwords.words('turkish'))

# Metin Temizleme Fonksiyonu
def metin_temizle(metin):
    metin = metin.lower()
    metin = re.sub(r'[^a-zçğıöşü\s]', '', metin)
    metin = re.sub(r'\s+', ' ', metin).strip()
    metin = ' '.join([kelime for kelime in metin.split() if kelime not in stop_words])
    return metin

# Dosya Okuma Fonksiyonu
def dosyalari_oku(dosya_yolu):
    try:
        with open(dosya_yolu, 'r', encoding='utf-8') as file:
            raporlar = file.read()
    except UnicodeDecodeError:
        with open(dosya_yolu, 'r', encoding='ISO-8859-1') as file:
            raporlar = file.read()
    return raporlar

# Dosyaları Ayırma Fonksiyonu
def dosyalari_ayir(dizin_yolu, train_count, val_count, test_count):
    dosyalar = os.listdir(dizin_yolu)
    np.random.shuffle(dosyalar)

    train_dosyalar = dosyalar[:train_count]
    val_dosyalar = dosyalar[train_count:train_count + val_count]
    test_dosyalar = dosyalar[train_count + val_count:train_count + val_count + test_count]

    return train_dosyalar, val_dosyalar, test_dosyalar

# BIRADS Klasör Yolları
birads_klasor_yollari = [
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS1",
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS2",
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS3",
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS4",
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/2000raporBIRADSgruplanmış/BIRADS5"
]
# Sınıf başına veri sayıları
birads_sayilari = [405, 518, 615, 721, 759]

# Eğitim, doğrulama ve test oranları
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Eğitim, doğrulama ve test setleri
X_train_raporlar = []
X_val_raporlar = []
X_test_raporlar = []
y_train = []
y_val = []
y_test = []

# Dosyaları sınıflara göre ayırma ve okuma
for i, dizin_yolu in enumerate(birads_klasor_yollari):
    birads_sayi = birads_sayilari[i]
    train_count = int(birads_sayi * train_ratio)
    val_count = int(birads_sayi * val_ratio)
    test_count = birads_sayi - train_count - val_count

    train_files, val_files, test_files = dosyalari_ayir(dizin_yolu, train_count, val_count, test_count)

    # Eğitim seti için
    for dosya in train_files:
        rapor = dosyalari_oku(os.path.join(dizin_yolu, dosya))
        X_train_raporlar.append(metin_temizle(rapor))
        y_train.append(i + 1)

    # Doğrulama seti için
    for dosya in val_files:
        rapor = dosyalari_oku(os.path.join(dizin_yolu, dosya))
        X_val_raporlar.append(metin_temizle(rapor))
        y_val.append(i + 1)

    # Test seti için
    for dosya in test_files:
        rapor = dosyalari_oku(os.path.join(dizin_yolu, dosya))
        X_test_raporlar.append(metin_temizle(rapor))
        y_test.append(i + 1)

# TF-IDF Vektörleştirme
vectorizer = TfidfVectorizer(min_df=1)
X_train_tfidf = vectorizer.fit_transform(X_train_raporlar)
X_val_tfidf = vectorizer.transform(X_val_raporlar)
X_test_tfidf = vectorizer.transform(X_test_raporlar)

# SMOTE uygulama (yalnızca eğitim setine)
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)


In [61]:
# Model Eğitimi ve Değerlendirme
model = MultinomialNB()
model.fit(X_train_resampled, y_train_resampled)

# Doğrulama seti değerlendirmesi
y_val_pred = model.predict(X_val_tfidf)
print("Doğrulama Seti Accuracy:", accuracy_score(y_val, y_val_pred))
print("Doğrulama Seti Classification Report:\n", classification_report(y_val, y_val_pred))

# Test seti değerlendirmesi
y_test_pred = model.predict(X_test_tfidf)
print("Test Seti Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Seti Classification Report:\n", classification_report(y_test, y_test_pred))


Doğrulama Seti Accuracy: 0.8494983277591973
Doğrulama Seti Classification Report:
               precision    recall  f1-score   support

           1       0.86      0.95      0.90        40
           2       0.69      0.78      0.73        51
           3       0.86      0.80      0.83        61
           4       0.97      0.78      0.86        72
           5       0.87      0.95      0.90        75

    accuracy                           0.85       299
   macro avg       0.85      0.85      0.85       299
weighted avg       0.86      0.85      0.85       299

Test Seti Accuracy: 0.8333333333333334
Test Seti Classification Report:
               precision    recall  f1-score   support

           1       0.93      0.95      0.94        41
           2       0.67      0.74      0.70        53
           3       0.82      0.81      0.81        62
           4       0.87      0.75      0.81        73
           5       0.88      0.94      0.91        77

    accuracy                 

In [67]:
import json
import os

# JSON dosyasını yükleme
def json_dosyasi_oku(json_dosya_yolu):
    with open(json_dosya_yolu, 'r', encoding='utf-8') as file:
        veri = json.load(file)
    return veri

# JSON dosyasına yazma
def json_dosyasi_yaz(json_dosya_yolu, veri):
    with open(json_dosya_yolu, 'w', encoding='utf-8') as file:
        json.dump(veri, file, ensure_ascii=False, indent=4)

# Test raporlarını tahmin etme ve sonuçları JSON'a yazma
def birads_tahmini_yap(json_dosya_yolu, model, vectorizer):
    # JSON dosyasını oku
    veri = json_dosyasi_oku(json_dosya_yolu)

    # Her raporu temizle ve tahmin
    for rapor in veri['tahminler']:
        metin = metin_temizle(rapor['text'])  # Raporu temizle
        tfidf_metin = vectorizer.transform([metin])  # TF-IDF dönüşümü
        tahmin = model.predict(tfidf_metin)  # Model ile tahmin yap
        rapor['cats'] = int(tahmin[0])  # Tahmini 'cats' kısmına ekle

    # Yeni JSON dosyasına yazma
    json_dosyasi_yaz('tahminli_' + os.path.basename(json_dosya_yolu), veri)

# JSON dosyası ile model ve vectorizer'ı kullanarak tahmin
json_dosya_yolu = 'C:/Users/BUSRA/Documents/NLP-Teknofest24/nlpFİNAL.json'
birads_tahmini_yap(json_dosya_yolu, model, vectorizer)


VARLIK İSMİ ÇIKARIMI BURDAN BAŞLIYOR


In [38]:
import json

file_paths = [
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/1/all.jsonl",
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/2/all.jsonl",
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/3/all.jsonl",
    "C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/4/all.jsonl"
]

combined_data = []

for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Satırı JSON olarak ayrıştırma ve listeye ekleme
                data = json.loads(line)
                combined_data.append(data)
            except json.JSONDecodeError:
                print(f"Geçersiz JSON satırı atlandı: {line}")

# Birleştirilmiş veriyi yeni bir JSONL dosyasına yazdırma
output_file_path = "C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/combined_all.jsonl"
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for entry in combined_data:
        # Her bir JSON nesnesini satır olarak yazma
        output_file.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"Dosyalar başarıyla {output_file_path} konumunda birleştirildi ve kontrol edildi.")


Dosyalar başarıyla C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/combined_all.jsonl konumunda birleştirildi ve kontrol edildi.


In [ ]:
import json

data = []
with open('C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/combined_all.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))  # Her satırı JSON olarak yükle ve listeye ekle

# Entite etiketlerini görselleştirme
def visualize_labeled_text(entry):
    labeled_text = entry['text'] 
    output_text = ""
    labels = entry['label']

    # "SONUÇ" veya "Sonuç" veya "SONUC" ile başlayan metni çıkartmak için kontrol
    if any(sonuc in labeled_text for sonuc in ["SONUÇ", "Sonuç", "SONUC"]):
        for sonuc in ["SONUÇ", "Sonuç", "SONUC"]:
            if sonuc in labeled_text:
                labeled_text = labeled_text.split(sonuc)[0]

    last_index = 0
    combined_labels = []

    # Aynı türdeki entiteleri birleştirmek için güncellenmiş yaklaşım
    for start, end, label in labels:
        # IMPRESSION etiketlerini atla
        if label == "IMPRESSION":
            continue

        # Eğer önceki entite ile devamlılık varsa ve türler aynı ise, entiteleri birleştir
        if combined_labels and combined_labels[-1][2] == label:
            # Aynı entite türü için mevcut birleşik entitenin bitişi ile yenisinin başlangıcı arasında boşluk, noktalama, vb. olabilir
            combined_labels[-1] = (combined_labels[-1][0], max(combined_labels[-1][1], end), label)
        else:
            combined_labels.append((start, end, label))

    # Birleştirilmiş etiketlerle metin oluşturma
    for start, end, label in combined_labels:
        # Etiketli metin kısmını oluşturma
        output_text += labeled_text[last_index:start]  
        output_text += f"{labeled_text[start:end]} [{label}]"  
        last_index = end

    # Etiketlerden sonraki kalan metni ekleme
    output_text += labeled_text[last_index:]

    return output_text

# Her bir örnek için etiketli metinleri görüntüleme
for entry in data:
    print(f"Metin ID: {entry['id']}")
    labeled_text = visualize_labeled_text(entry)
    print(labeled_text)
    print("\n" + "-"*50 + "\n")


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


In [23]:
import json

# Etiketlenmiş veri dosyasını oku
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Etiketleri temizleme ve boşlukları kaldırma
def clean_labels(data):
    cleaned_data = []
    for entry in data:
        text = entry['text']
        cleaned_labels = []
        for start, end, label in entry['label']:
            entity = text[start:end]
            # Öncül ve ardıl boşlukları kaldırma
            new_entity = entity.strip()
            new_start = start + (len(entity) - len(entity.lstrip()))
            new_end = end - (len(entity) - len(entity.rstrip()))
            cleaned_labels.append((new_start, new_end, label))
        cleaned_data.append({'id': entry['id'], 'text': text, 'label': cleaned_labels})
    return cleaned_data

# Güncellenmiş veriyi kaydetme
def save_cleaned_data(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f:
        for entry in data:
            json.dump(entry, f, ensure_ascii=False)
            f.write('\n')

def main():
    file_path = 'C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/combined_all.jsonl'
    cleaned_file_path = 'C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/combined_all_cleaned.jsonl'

    data = load_data(file_path)
    cleaned_data = clean_labels(data)

    save_cleaned_data(cleaned_file_path, cleaned_data)
    print("Temizlenmiş veri dosyası kaydedildi:", cleaned_file_path)

if __name__ == "__main__":
    main()


Temizlenmiş veri dosyası kaydedildi: C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/combined_all_cleaned.jsonl


In [ ]:
import spacy
from spacy.training import Example

def load_cleaned_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Veri hazırlama
def prepare_data(data):
    examples = []
    for entry in data:
        text = entry['text']
        ents = [(start, end, label) for start, end, label in entry['label']]
        examples.append((text, {"entities": ents}))
    return examples

# Model eğitimi
def train_ner_model(train_data):
    nlp = spacy.blank("en")
    ner = nlp.add_pipe("ner", last=True)

    # Etiketleri ekle
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # Eğitici oluştur ve eğitime başla
    optimizer = nlp.begin_training()
    for i in range(20):  # Eğitim döngüsü sayısı
        losses = {}
        for text, annotations in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], sgd=optimizer, losses=losses, drop=0.35)
        print(f"Losses at iteration {i}: {losses}")

    # Eğitilmiş modeli kaydet
    nlp.to_disk("ner_model")

# Ana fonksiyon
def main():
    cleaned_file_path = 'C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/combined_all_cleaned.jsonl'

    data = load_cleaned_data(cleaned_file_path)
    train_data = prepare_data(data)
    train_ner_model(train_data)
    print("Model eğitildi ve kaydedildi.")

if __name__ == "__main__":
    main()


In [36]:
import spacy
from spacy.training import Example

# Eğitilmiş modeli yükle
def load_model(model_path):
    nlp = spacy.load(model_path)
    return nlp

# Metin üzerinde varlık çıkarımı yap
def predict_entities(nlp, text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Veri hazırlama (Example nesneleri oluşturma)
def prepare_examples(data, nlp):
    examples = []
    for entry in data:
        text = entry['text']
        ents = [(start, end, label) for start, end, label in entry['label']]
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, {"entities": ents})
        examples.append(example)
    return examples

# Modeli değerlendir
def evaluate_model(nlp, data):
    examples = prepare_examples(data, nlp)
    scorer = nlp.evaluate(examples)
    print(f"Precision: {scorer['ents_p']:.2f}")
    print(f"Recall: {scorer['ents_r']:.2f}")
    print(f"F1-Score: {scorer['ents_f']:.2f}")

# Ana fonksiyon
def main():
    model_path = "ner_model"
    nlp = load_model(model_path)

    # Test metni
    test_text = "Örnek bir metin, burada varlık çıkarımı yapılacak."
    entities = predict_entities(nlp, test_text)
    print("Çıkarılan varlıklar:", entities)

    # Modeli değerlendirme
    cleaned_file_path = 'C:/Users/BUSRA/Documents/NLP-Teknofest24/NLPJSON/combined_all_cleaned.jsonl'
    data = load_cleaned_data(cleaned_file_path)
    evaluate_model(nlp, data)

if __name__ == "__main__":
    main()


Çıkarılan varlıklar: [('bir metin', 'OBS-PRESENT'), ('burada varlık çıkarımı', 'OBS-PRESENT')]
Precision: 0.77
Recall: 0.71
F1-Score: 0.74


In [58]:
import json
import spacy

# Eğitilmiş modeli yükle
def load_model(model_path):
    nlp = spacy.load(model_path)
    return nlp

# Metin üzerinde varlık çıkarımı yap
def predict_entities(nlp, text):
    doc = nlp(text)
    entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
    return entities

# Aynı türdeki bitişik varlıkları birleştir
def merge_adjacent_entities(entities):
    if not entities:
        return entities
    
    merged_entities = []
    current_start, current_end, current_label = entities[0]
    
    for start, end, label in entities[1:]:
        # Ardışık ve aynı etiketli ve arada boşluk/bağlaç varsa
        if label == current_label and start <= current_end + 1:  
            current_end = end  # Bitiş noktasını güncelle
        else:
            merged_entities.append((current_start, current_end, current_label))
            current_start, current_end, current_label = start, end, label
    
    # Son kalan varlığı ekle
    merged_entities.append((current_start, current_end, current_label))
    
    return merged_entities

# Test dosyasını yükle
def load_test_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Test dosyasını işle ve varlık çıkarım sonuçlarını entities anahtarına ekle
def process_test_data(nlp, test_data):
    for entry in test_data['tahminler']:  # "tahminler" altındaki verileri işliyoruz
        text = entry.get('text', '')  # Her entry'den text'i alın
        if text:
            entities = predict_entities(nlp, text)  # Model ile varlık çıkarımı yap
            merged_entities = merge_adjacent_entities(entities)  # Aynı etiketli bitişik varlıkları birleştir
            entry['entities'] = merged_entities  # Birleştirilmiş varlıkları entities kısmına yaz
    return test_data

# Sonuçları yeni bir dosyaya kaydet
def save_results(output_file_path, data):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# Ana fonksiyon
def main():
    model_path = "ner_model"  # Eğitilmiş model yolu
    test_file_path = "C:/Users/BUSRA/Documents/NLP-Teknofest24/nlp_test_dataset (2)/nlp_test_dataset.json"  # Test dosyanızın yolu
    output_file_path = "C:/Users/BUSRA/Documents/NLP-Teknofest24/tahminli_nlp_demo_dataset_with_entities_merged.json"  # Sonuçların kaydedileceği dosya

    # Modeli yükle
    nlp = load_model(model_path)

    # Test verilerini yükle
    test_data = load_test_data(test_file_path)

    # Test verilerini işle ve varlık çıkarımı yap
    processed_data = process_test_data(nlp, test_data)

    # Sonuçları yeni bir dosyaya kaydet
    save_results(output_file_path, processed_data)

    print(f"Varlık çıkarımı tamamlandı ve sonuçlar {output_file_path} dosyasına kaydedildi.")

if __name__ == "__main__":
    main()


Varlık çıkarımı tamamlandı ve sonuçlar C:/Users/BUSRA/Documents/NLP-Teknofest24/tahminli_nlp_demo_dataset_with_entities_merged.json dosyasına kaydedildi.


In [59]:
import json

# Load the JSON file
file_path = 'C:/Users/BUSRA/Documents/NLP-Teknofest24/tahminli_nlp_demo_dataset_with_entities_merged.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Update the "kunye" section
data['kunye']['takim_adi'] = "Diagnos-ai"
data['kunye']['takim_id'] = "344193"
data['kunye']['aciklama'] = "ikinci aşama çıktısı"

# Save the updated JSON back to a new file
output_file_path = 'C:/Users/BUSRA/Documents/NLP-Teknofest24/nlpFİNAL.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

output_file_path


'C:/Users/BUSRA/Documents/NLP-Teknofest24/nlpFİNAL.json'

In [69]:
import json
import spacy

# Load the trained model
def load_model(model_path):
    nlp = spacy.load(model_path)
    return nlp

# Perform entity recognition on text
def predict_entities(nlp, text):
    doc = nlp(text)
    entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
    return entities

# Merge adjacent entities with the same label
def merge_adjacent_entities(entities):
    if not entities:
        return entities
    
    merged_entities = []
    current_start, current_end, current_label = entities[0]
    
    for start, end, label in entities[1:]:
        # If consecutive and same-labeled entities are adjacent
        if label == current_label and start <= current_end + 1:  
            current_end = end  # Update the end position
        else:
            merged_entities.append((current_start, current_end, current_label))
            current_start, current_end, current_label = start, end, label
    
    # Append the last remaining entity
    merged_entities.append((current_start, current_end, current_label))
    
    return merged_entities

# Load the test data file
def load_test_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Process the test data and add entity extraction results to the "entities" field
def process_test_data(nlp, test_data):
    for entry in test_data['tahminler']:  # Process entries under "tahminler"
        text = entry.get('text', '')  # Get the text for each entry
        if text:
            entities = predict_entities(nlp, text)  # Extract entities with the model
            merged_entities = merge_adjacent_entities(entities)  # Merge adjacent entities of the same label
            
            # Format entities according to the example in the image
            formatted_entities = [{"start": start, "end": end, "label": label} for start, end, label in merged_entities]
            entry['entities'] = formatted_entities  # Assign formatted entities back to the entry

    return test_data

# Save results to a new file
def save_results(output_file_path, data):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# Main function
def main():
    model_path = "ner_model"  # Path to your trained model
    test_file_path = "C:/Users/BUSRA/Documents/NLP-Teknofest24/nlp_demo_dataset.json" # Path to your test dataset
    output_file_path = "C:/Users/BUSRA/Documents/NLP-Teknofest24/tahminli_nlp_demo_dataset_with_entities_merged.json"  # Path to save results

    # Load the model
    nlp = load_model(model_path)

    # Load the test data
    test_data = load_test_data(test_file_path)

    # Process test data and perform entity extraction
    processed_data = process_test_data(nlp, test_data)

    # Save the results to a new file
    save_results(output_file_path, processed_data)

    print(f"Entity extraction complete. Results saved to {output_file_path}.")

if __name__ == "__main__":
    main()


Entity extraction complete. Results saved to C:/Users/BUSRA/Documents/NLP-Teknofest24/tahminli_nlp_demo_dataset_with_entities_merged.json.
